# 타이핑 연습게임-기록 저장
- 타이핑 게임을 수행
- 게임 결과(정답 개수, 소요 시간 등)를 SQLite 데이터베이스에 저장
- 정답 여부에 따라 소리를 출력하여 사용자에게 피드백을 제공

In [ ]:
############################################################################################################
# 미션 : 타자 연습 게임 프로그램 개발
# - Python을 사용하여 영어 단어 타자 연습 게임을 개발하세요.
# - 게임 결과를 SQLite 데이터베이스에 저장하고, 정답 여부에 따라 피드백(소리와 메시지)을 제공합니다.

# 요구사항:
# 1. 데이터베이스 생성:
#    - 데이터베이스 파일 이름: `records.db`
#    - 테이블 이름: `records`
#    - 테이블 스키마:
#      - id: INTEGER, PRIMARY KEY, AUTOINCREMENT (자동으로 증가하는 고유 식별자)
#      - cor_cnt: INTEGER (정답 개수)
#      - record: TEXT (총 소요 시간)
#      - regdate: TEXT (기록된 날짜와 시간)

# 2. 단어 로드:
#    - `data/word.txt` 파일에서 영어 단어를 읽어 리스트에 저장하세요.
#    - 단어가 총 몇 개인지 출력하여 확인하세요.

# 3. 게임 시작:
#    - 사용자가 Enter 키를 누르면 게임이 시작됩니다.

# 4. 게임 진행:
#    - 랜덤으로 단어를 선택하여 출력하고, 사용자는 해당 단어를 정확히 입력해야 합니다.
#    - 총 5개의 문제를 출제하며, 정답 여부를 확인합니다.
#    - 정답이면 "Pass!" 메시지와 함께 정답 소리를 재생하고, 정답 개수를 증가시킵니다.
#    - 오답이면 "Wrong!" 메시지와 함께 오답 소리를 재생합니다.

# 5. 게임 종료:
#    - 게임이 종료되면 사용자의 정답 개수와 총 소요 시간을 계산합니다.
#    - 정답이 3개 이상이면 "합격", 그렇지 않으면 "불합격" 메시지를 출력하세요.

# 6. 결과 저장:
#    - SQLite 데이터베이스에 게임 결과(정답 개수, 소요 시간, 기록 시각)를 저장하세요.

# 7. 최종 결과 출력:
#    - 게임의 총 소요 시간과 정답 개수를 출력하세요.

# 결과:
# - Python 코드를 실행하면 단어 타자 연습 게임이 시작됩니다.
# - 정답 여부에 따라 소리가 출력되고, 정답 개수를 기록합니다.
# - 게임 결과가 콘솔에 출력되며, SQLite 데이터베이스(`records.db`)에 저장됩니다.

# 조건:
# - Python의 `sqlite3` 모듈과 `winsound` 모듈을 사용합니다.
# - 단어 파일(`word.txt`)은 영어 단어로 구성되어야 합니다.
# - 커서와 데이터베이스 연결을 적절히 종료하여 리소스 누수를 방지하세요.

# 주의:
# - `word.txt`, `good.wav`, `bad.wav` 파일의 경로를 정확히 설정하세요.
# - 소리가 제대로 재생되지 않을 경우 파일 경로를 다시 확인하세요.
# - 데이터베이스에 기록된 데이터가 올바르게 저장되었는지 확인하세요.
############################################################################################################


In [1]:
import sqlite3
import winsound
import time
import random
import datetime
import os

# 1. 단어 파일 경로 설정
word_file_path = 'files/word.txt'

# 2. 소리 파일 경로 설정
good_sound_path = 'data/good.wav'
bad_sound_path = 'data/bad.wav'

# 3. 데이터베이스 파일 경로 설정
db_path = 'records.db'

# 4. 단어 리스트 불러오기
words = []
try:
    with open(word_file_path, 'r') as f:
        for c in f:
            words.append(c.strip())
except FileNotFoundError:
    print(f"Error: 단어 파일을 찾을 수 없습니다. 경로를 확인하세요 -> {word_file_path}")
    exit()

print("단어 리스트 로드 완료. 단어 개수:", len(words))

# 5. 게임 시작 대기
input("Ready? Press Enter Key!")

# 6. 게임 초기화
n = 1
cor_cnt = 0
start = time.time()

# 7. 게임 실행 (5회 반복)
while n <= 5:
    random.shuffle(words)
    q = random.choice(words)

    print(f"\n* Question # {n}")
    print(q)

    x = input("타이핑: ")

    if q.strip() == x.strip():
        print("Pass!")
        winsound.PlaySound(good_sound_path, winsound.SND_FILENAME)  # 정답 소리
        cor_cnt += 1
    else:
        print("Wrong!")
        winsound.PlaySound(bad_sound_path, winsound.SND_FILENAME)  # 오답 소리

    n += 1

# 8. 게임 종료
end = time.time()
et = end - start
et = format(et, ".3f")  # 소수점 3자리로

# 9. 결과 출력
print("\n----- 결과 -----")
if cor_cnt >= 3:
    print("결과: 합격 🎉")
else:
    print("결과: 불합격 😢")
print(f"게임 시간: {et} 초, 정답 개수: {cor_cnt}개")

# 10. 데이터베이스 저장
conn = sqlite3.connect(db_path)
cur = conn.cursor()

try:
    # 테이블 생성
    cur.execute('''
        CREATE TABLE IF NOT EXISTS records (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cor_cnt INTEGER,
            record TEXT,
            regdate TEXT
        )
    ''')

    # 데이터 삽입
    cur.execute('''
        INSERT INTO records (cor_cnt, record, regdate) 
        VALUES (?, ?, ?)
    ''', (cor_cnt, et, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

    conn.commit()

except Exception as e:
    print("Error 발생:", e)
    conn.rollback()

finally:
    cur.close()
    conn.close()

print("게임 기록이 데이터베이스에 저장되었습니다.")


단어 리스트 로드 완료. 단어 개수: 9859

* Question # 1
creative
Pass!

* Question # 2
colour
Wrong!

* Question # 3
marcus
Pass!

* Question # 4
wet
Pass!

* Question # 5
michigan
Pass!

----- 결과 -----
결과: 합격 🎉
게임 시간: 17.848 초, 정답 개수: 4개
게임 기록이 데이터베이스에 저장되었습니다.
